<H1>HC in Cross-Domain Authorship Attribution Challenge</H1>
Use this notebook to replicate the results reporter in the paper  </br>
<ul>
    [1] <a href = https://arxiv.org/abs/1911.01208>
    Kipnis, A., ``Higher Criticism for Discriminating Word-Frequency Tables and Testing Authorship'', 2019
    </a>
</ul>

- Use HC-based test to attribute authorship in the PAN2018 cross-domain authorship attribution challenge https://pan.webis.de/clef18/pan18-web/author-identification.html#cross-domain
- Only the English part (problems 1-4) of this challenge is considered. 
- We use a lemmatized version of the data obtained using the Stanford CoreNLP (https://stanfordnlp.github.io/CoreNLP/)



In [2]:
import pandas as pd
import numpy as np
import os
import re
import codecs
from tqdm import tqdm

#import auxiliary functions for python
import sys
sys.path.append('../')
from AuthAttLib import *

<H2>Load Data</H2>
The data below was obtained by lemmatizing the original challenge test data using the Stanford CoreNLP lemmatizer https://stanfordnlp.github.io/CoreNLP/. 

In [5]:
#load data PAN2018 (from lemmatized) 
raw_data = pd.read_csv("./Data/PAN2018_lemmatized.csv")
raw_data.loc[:,'lem_text'] = raw_data.loc[:,'lem_text'] # + ' ' + raw_data.loc[:,'POS']
data = raw_data.filter(['dataset', 'author', 'doc_id'])

data.loc[:,'split'] = 'train'
data.loc[data.doc_id.str.find('test')>-1,'split'] = 'test'

Ignore proper names, numbers, and some pronouns: 

In [6]:
#remove proper names
data.loc[:,'text'] = raw_data.apply(
    lambda r : " ".join([w[0] for w in zip(r['lem_text'].split(),r['POS'].split()) if w[1] != 'PROPN']), axis=1)


<H2>Multi-author model</H2>

- For each problem in the challenge, train a model and evaluate it over a test set. <br>
- The following implementation opt not to use the UNNKOWN option, hence the recall is always 100%

In [10]:

lo_problem = pd.unique(data_train['dataset'])

lo_F1 = []
lo_acc = []

for prob in tqdm(lo_problem[:4]) :
    data_prob = data[data['dataset'] == prob]
    
    data_train = data_prob[data_prob['split'] == 'train']
    #compute model for each problem:
    model = AuthorshipAttributionMulti(data_prob, 
                                       vocab_size = 1500,  #uses 3000 most frequent ngram
                                       stbl = True,  #type of HC statistic
                                      ngram_range = (1,3), #mono-, bi-, and tri- grams
                                       flat=True  # compress counts in each corpus (faster)
                                                )
    
    #attribute test documents:
    print("Evaluate on test dataset:")
    data_test = data_prob[data_prob['split'] == 'test']
    lo_test_docs = pd.unique(data_test.doc_id)
    df = pd.DataFrame() #save results in this dataframe

    for doc in tqdm(lo_test_docs) :
        sm = data_test[data_test.doc_id == doc]

        pred,_ = model.predict(sm.text.values[0], unk_thresh = 1e6) 
                # can use 'unk_thresh' to get '<UNK>' instead of the name 
                # of the corpus with smallest HC in the case when the smallest
                # HC is above 'unk_thresh'. 

        auth = sm.author.values[0]
        df = df.append({'doc_id' : doc,
                   'author' : auth,
                   'predicted' : pred,
                  }, ignore_index = True)


    # evaluate accuracy and F1 score
    df_r = df[df.predicted != '<UNK>']
    recall = len(df_r) / len(df)
    acc = np.mean((df_r.predicted == df_r.author).values)
    
    print("problem = {}".format(prob))
    print("recall = {}".format(recall))
    print("accuracy = {}".format(acc))
    print("F1 = {}".format(2*recall*acc / (recall + acc)))
    lo_F1 += [2*recall*acc / (recall + acc)]
    lo_acc += [acc]

#prob1: |W| = 3000, ng = (1,3) --> F1 = 0.6017, acc = 0.43 
#       |W| = 1500, ng = (1,3) --> F1 = 0.672, acc = 0.5063
#prob2: |W| = 3000, ng = (1,3) --> F1 = 0.666, acc = 0.5 
#       |W| = 1500, ng = (1,3) --> F1 = 0.67857, acc = 0.5135
#prob3: |W| = 3000, ng = (1,3) --> F1 = 0.8732, acc = 0.775
#       |W| = 1500, ng = (1,3) --> F1 = 0.80597014, acc = 0.675
#prob4: |W| = 3000, ng = (1,3) --> F1 = 0.857, acc = 0.75
#       |W| = 1500, ng = (1,3) --> F1 = 0.933, acc = 0.857


  0%|          | 0/4 [00:00<?, ?it/s]

	 Creating author-model for candidate00001 using 1500 features...
		found 7 documents and 8588 relevant tokens.
	 Creating author-model for candidate00002 using 1500 features...
		found 7 documents and 8498 relevant tokens.
	 Creating author-model for candidate00003 using 1500 features...
		found 7 documents and 7619 relevant tokens.
	 Creating author-model for candidate00004 using 1500 features...
		found 7 documents and 8383 relevant tokens.
	 Creating author-model for candidate00005 using 1500 features...
		found 7 documents and 7809 relevant tokens.
	 Creating author-model for candidate00006 using 1500 features...
		found 7 documents and 8443 relevant tokens.
	 Creating author-model for candidate00007 using 1500 features...
		found 7 documents and 7806 relevant tokens.
	 Creating author-model for candidate00008 using 1500 features...
		found 7 documents and 7433 relevant tokens.
	 Creating author-model for candidate00009 using 1500 features...
		found 7 documents and 7846 relevant 


  0%|          | 0/79 [00:00<?, ?it/s]

		found 7 documents and 8249 relevant tokens.
Evaluate on test dataset:



 25%|██▌       | 1/4 [09:20<28:01, 560.59s/it]]

...

problem = PAN-problem00003
recall = 1.0
accuracy = 0.675
F1 = 0.8059701492537313
	 Creating author-model for candidate00001 using 1500 features...
		found 7 documents and 7858 relevant tokens.
	 Creating author-model for candidate00002 using 1500 features...
		found 7 documents and 7814 relevant tokens.
	 Creating author-model for candidate00003 using 1500 features...
		found 7 documents and 8925 relevant tokens.
	 Creating author-model for candidate00004 using 1500 features...



  0%|          | 0/16 [00:00<?, ?it/s]

		found 7 documents and 7056 relevant tokens.
	 Creating author-model for candidate00005 using 1500 features...
		found 7 documents and 7376 relevant tokens.
Evaluate on test dataset:



100%|██████████| 4/4 [18:42<00:00, 288.43s/it]]

problem = PAN-problem00004
recall = 1.0
accuracy = 0.875
F1 = 0.9333333333333333


<h2>Multi-author with head-to-head comparisons</h2>

Compare each pair of corpora. Use only distinguishing features of two corpora in testing. Attribute tested docuement to whichever corpus has most number of wins in all pairwise comparisons.

In [13]:

lo_problem = pd.unique(data_train['dataset'])

lo_F1 = []
lo_acc = []

for prob in tqdm(lo_problem[3:4]) :
    data_prob = data[data['dataset'] == prob]
    data_train = data_prob[data_prob['split'] == 'train']
    
    #compute model for each problem:
    model = AuthorshipAttributionMultiBinary(data_prob, 
                                       vocab_size = 100,  #uses 3000 most frequent ngram
                                       stbl = True,  #type of HC statistic
                                      ngram_range = (1,3), #mono-, bi-, and tri- grams
                                             reduce_features=True
                                                )
    
    #attribute test documents:
    data_test = data_prob[data_prob['split'] == 'test']
    lo_test_docs = pd.unique(data_test.doc_id)
    df = pd.DataFrame() #save results in this dataframe

    for doc in tqdm(lo_test_docs) :
        sm = data_test[data_test.doc_id == doc]

        pred = model.predict(sm.text.values[0], method = 'HC') 
                # can use 'unk_thresh' to get '<UNK>' instead of the name 
                # of the corpus with smallest HC in the case when the smallest
                # HC is above 'unk_thresh'. 

        auth = sm.author.values[0]
        df = df.append({'doc_id' : doc,
                   'author' : auth,
                   'predicted' : pred,
                  }, ignore_index = True)


    # evaluate accuracy and F1 score
    df_r = df[df.predicted != '<UNK>']
    recall = len(df_r) / len(df)
    acc = np.mean((df_r.predicted == df_r.author).values)
    
    print("problem = {}".format(prob))
    print("recall = {}".format(recall))
    print("accuracy = {}".format(acc))
    print("F1 = {}".format(2*recall*acc / (recall + acc)))
    lo_F1 += [2*recall*acc / (recall + acc)]
    lo_acc += [acc]

#prob4: F1 = 0.72, acc = 0.5625, |W| = 100, ng = (1,3)








  0%|          | 0/1 [00:00<?, ?it/s]






  0%|          | 0/10 [00:00<?, ?it/s]

Found 10 author-pairs
MultiBinaryAuthorModel: Creating model for candidate00001 vs candidate00002
	 Creating author-model for candidate00001 using 1774 features
		found 7 documents and 5715 relevant tokens
	 Creating author-model for candidate00002 using 1774 features









 10%|█         | 1/10 [00:03<00:29,  3.28s/it]

		found 7 documents and 5549 relevant tokens
MultiBinaryAuthorModel: Creating model for candidate00001 vs candidate00003
	 Creating author-model for candidate00001 using 1811 features
		found 7 documents and 5715 relevant tokens
	 Creating author-model for candidate00003 using 1811 features









 20%|██        | 2/10 [00:06<00:25,  3.17s/it]

...

		found 7 documents and 5019 relevant tokens
MultiBinaryAuthorModel: Creating model for candidate00004 vs candidate00005
	 Creating author-model for candidate00004 using 2170 features
		found 7 documents and 6041 relevant tokens
	 Creating author-model for candidate00005 using 2170 features









100%|██████████| 10/10 [00:33<00:00,  3.63s/it]






  0%|          | 0/16 [00:00<?, ?it/s]

		found 7 documents and 5019 relevant tokens









  6%|▋         | 1/16 [00:07<01:50,  7.40s/it]






 12%|█▎        | 2/16 [00:12<01:32,  6.59s/it]






 19%|█▉        | 3/16 [00:16<01:17,  5.99s/it]






 25%|██▌       | 4/16 [00:21<01:06,  5.57s/it]






 31%|███▏      | 5/16 [00:26<00:59,  5.42s/it]






 38%|███▊      | 6/16 [00:31<00:53,  5.34s/it]






 44%|████▍     | 7/16 [00:36<00:46,  5.12s/it]






 50%|█████     | 8/16 [00:41<00:41,  5.13s/it]






 56%|█████▋    | 9/16 [00:46<00:35,  5.12s/it]






 62%|██████▎   | 10/16 [00:50<00:29,  4.96s/it]






 69%|██████▉   | 11/16 [00:55<00:24,  4.91s/it]






 75%|███████▌  | 12/16 [01:00<00:19,  4.86s/it]






 81%|████████▏ | 13/16 [01:05<00:14,  4.85s/it]






 88%|████████▊ | 14/16 [01:09<00:09,  4.78s/it]






 94%|█████████▍| 15/16 [01:14<00:04,  4.85s/it]






100%|██████████| 16/16 [01:19<00:00,  4.89s/it]





100%|██████████| 1/1 [01:53<00:00, 113.07s/it]

problem = PAN-problem00004
recall = 1.0
accuracy = 0.5625
F1 = 0.72
